In [2]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from utils import GLOBAL, functions

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [4]:
#Create DFs
yearly_stats_df = pd.read_csv(GLOBAL.STATS_2020)

In [5]:
#Calculate Fantasy Scoring for all formats, Usage, Usage/G, and Pts/G (by format)

#Fantasy score
yearly_stats_df['FantasyPointsPPR'] = functions.scoringCalculator(
    yearly_stats_df['Rec'], yearly_stats_df['ReceivingYds'], yearly_stats_df['ReceivingTD'], yearly_stats_df['FumblesLost'], yearly_stats_df['RushingYds'], yearly_stats_df['RushingTD'], yearly_stats_df['PassingYds'], yearly_stats_df['PassingTD'], yearly_stats_df['Int'], GLOBAL.PPR_SCORING)

yearly_stats_df['FantasyPointsHalfPPR'] = functions.scoringCalculator(
    yearly_stats_df['Rec'], yearly_stats_df['ReceivingYds'], yearly_stats_df['ReceivingTD'], yearly_stats_df['FumblesLost'], yearly_stats_df['RushingYds'], yearly_stats_df['RushingTD'], yearly_stats_df['PassingYds'], yearly_stats_df['PassingTD'], yearly_stats_df['Int'], GLOBAL.HALF_PPR_SCORING)

yearly_stats_df['FantasyPoints'] = functions.scoringCalculator(
    yearly_stats_df['Rec'], yearly_stats_df['ReceivingYds'], yearly_stats_df['ReceivingTD'], yearly_stats_df['FumblesLost'], yearly_stats_df['RushingYds'], yearly_stats_df['RushingTD'], yearly_stats_df['PassingYds'], yearly_stats_df['PassingTD'], yearly_stats_df['Int'], GLOBAL.STANDARD_SCORING)


#Usage & Usage/G
yearly_stats_df['Usage'] = yearly_stats_df['RushingAtt'] + yearly_stats_df['PassingAtt'] + yearly_stats_df['Tgt']
yearly_stats_df['Usage/G'] = (yearly_stats_df['RushingAtt'] + yearly_stats_df['PassingAtt'] + yearly_stats_df['Tgt']) / yearly_stats_df['G']

#Pts/G
yearly_stats_df['StandardPoints/G'] = yearly_stats_df['FantasyPoints'] / yearly_stats_df['G']
yearly_stats_df['HalfPPRPoints/G'] = yearly_stats_df['FantasyPointsHalfPPR'] / yearly_stats_df['G']
yearly_stats_df['PPRPoints/G'] = yearly_stats_df['FantasyPointsPPR'] / yearly_stats_df['G']

In [5]:
#Create Usage rank & Fantasy Points Rank
yearly_stats_df['UsageRank'] = yearly_stats_df['Usage'].rank(ascending=False)

yearly_stats_df['StandardFantasyPointsRank'] = yearly_stats_df['FantasyPoints'].rank(ascending=False)
yearly_stats_df['HalfPPRFantasyPointsRank'] = yearly_stats_df['FantasyPointsHalfPPR'].rank(ascending=False)
yearly_stats_df['PPRFantasyPointsRank'] = yearly_stats_df['FantasyPointsPPR'].rank(ascending=False)

In [6]:
#Finding the Correlation based on Usage vs Scoring per format
x = yearly_stats_df['Usage']
y_std = yearly_stats_df['FantasyPoints']
y_half_ppr = yearly_stats_df['FantasyPointsHalfPPR']
y_ppr = yearly_stats_df['FantasyPointsPPR']

standard_correlation = functions.correlation(x, y_std)
half_ppr_correlation = functions.correlation(x, y_half_ppr)
ppr_correlation = functions.correlation(x, y_ppr)

In [7]:
#Use Machine Learning to test/train data

#Can change out Y var with above vars based on scoring format you want to use
X = x
Y = y_half_ppr

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [8]:
x_train[:20]

252     56
393     13
105    189
462     10
517      7
340     52
615      0
377     21
95     220
201     59
154     93
155    206
107    128
315     28
124    212
73      92
18     318
414      5
584      0
402     22
Name: Usage, dtype: int64